In [1]:
import numpy as np
import pandas as pd
import nibabel as nib

from scipy.ndimage import binary_dilation as dil
from scipy.ndimage import binary_erosion as ero

from skimage.metrics import structural_similarity as ssim
from skimage.filters import sobel

import glob as g

In [2]:
def ndil(x,n):
    for i in range(n):
        x = dil(x.copy())
    return x


def nero(x,n):
    for i in range(n):
        x = ero(x.copy())
    return x

In [3]:
#def norm(x):
#    return (x-x.min())/(x.max()-x.min())

def llog(a,b):
    r = np.zeros_like(a)
    mk = (a==0) | (b==0)
    r[~mk] = np.log(a[~mk]/b[~mk])
    return np.abs(r)

def kl(p, q):
    k = llog(p,q)
    return np.nansum(k) / np.prod(k.shape)

def skl(p,q):
    return kl(p,q) + kl(q,p)

def js(p,q):
    m = (p+q)/2
    return (kl(p,m) + kl(q,m))/2

In [4]:
def matrix(fname):
    a = nib.load(fname)
    return np.asanyarray(a.dataobj)

def borders(X, times = 3):
    segments = set(X.flatten())
    B = np.zeros(X.shape, dtype='bool')
    for s in segments:
        Xs = ( X == s)
        S = ndil(Xs, times) ^ nero(Xs, times)
        B = B | S
    return B

def gradient(X):
    return np.sqrt(sobel(X, axis=0)**2 + sobel(X, axis=1)**2 + sobel(X, axis=2)**2)

def err(X,Y):
    return 1 - ssim(X,Y)
    #return skl(X,Y)
    #return js(X,Y)

def errors(X, Y, Mask):
    Xm, Ym = X.copy(), Y.copy()
    Xm[~Mask] = 0
    Ym[~Mask] = 0
    return err(X, Y), err(Xm, Ym)

In [5]:
F = sorted(g.glob("*/*hq_reg*nii.gz"))
print(F)

['ANTs/SL_hq_reg.nii.gz', 'ANTs/SL_hq_reg_gauss.nii.gz', 'ANTs/SL_hq_reg_lancz.nii.gz', 'ANTs/SL_hq_reg_splines.nii.gz', 'ANTs/t1w_hq_reg.nii.gz', 'ANTs/t1w_hq_reg_gauss.nii.gz', 'ANTs/t1w_hq_reg_lancz.nii.gz', 'ANTs/t1w_hq_reg_splines.nii.gz', 'ANTs/walnut_hq_reg.nii.gz', 'ANTs/walnut_hq_reg_gauss.nii.gz', 'ANTs/walnut_hq_reg_lancz.nii.gz', 'ANTs/walnut_hq_reg_splines.nii.gz', 'FSL/SL_hq_reg.nii.gz', 'FSL/SL_hq_reg_splines.nii.gz', 'FSL/t1w_hq_reg.nii.gz', 'FSL/t1w_hq_reg_splines.nii.gz', 'FSL/walnut_hq_reg.nii.gz', 'FSL/walnut_hq_reg_splines.nii.gz', 'frees/SL_hq_reg.nii.gz', 'frees/SL_hq_reg_splines.nii.gz', 'frees/t1w_hq_reg.nii.gz', 'frees/t1w_hq_reg_splines.nii.gz', 'frees/walnut_hq_reg.nii.gz', 'frees/walnut_hq_reg_splines.nii.gz']


In [6]:
DF = []
for f in F:
    softw = f.split("/")[0]
    img = f.split("/")[-1].split("_")[0]
    interp =  f.split("_")[-1].replace(".nii.gz","")
    
    name = softw + "_" + img + "_" + interp
    print(name)
    
    A = matrix(f)
    B = matrix("data/" + img + "_hq.nii.gz")
    M = matrix("data/" + img + "_hq_seg.nii.gz")
    
    Mx = borders(M)
    
    e1, e2 = errors(A,B, Mx)
    vol_perc = np.sum(Mx)/np.prod(Mx.shape)*100
    
    G = gradient(B)
    grad_perc = np.sum(G[Mx])/np.sum(G) * 100
    
    
    print(e1,e2, e2/e1 * 100., grad_perc, vol_perc  )
    DF.append([name, e1, e2, e2/e1 * 100. , grad_perc, vol_perc])

ANTs_SL_reg


/anaconda/envs/py3k/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.


0.03715268229732638 0.031298470759810515 84.24282938533041 100.00000000000003 14.897966384887695
ANTs_SL_gauss
0.04700257261593155 0.03865199565371091 82.23378743445588 100.00000000000003 14.897966384887695
ANTs_SL_lancz
0.042189872278038165 0.035423055436926254 83.96103975732025 100.00000000000003 14.897966384887695
ANTs_SL_splines
0.04125464952329927 0.034413936045625704 83.41832119114197 100.00000000000003 14.897966384887695
ANTs_t1w_reg
0.06290074849006355 0.034254469761528794 54.45796844045502 89.20926009316376 18.886477820294783
ANTs_t1w_gauss
0.11126204083394187 0.051931667230716294 46.67509857043166 89.20926009316376 18.886477820294783
ANTs_t1w_lancz
0.08098500108864293 0.03388041495078897 41.83541951639271 89.20926009316376 18.886477820294783
ANTs_t1w_splines
0.10282575283648532 0.033517644499963306 32.5965466581737 89.20926009316376 18.886477820294783
ANTs_walnut_reg
0.044889049978321216 0.011283923432409138 25.1373629824169 71.41374012609639 14.23408707002457
ANTs_walnut_gau

In [7]:
df = pd.DataFrame(DF, columns = ["name", "error", "error_in_borders", "percentage", "perc_gradient", "perc_volume"])

In [8]:
df

,name,error,error_in_borders,percentage,perc_gradient,perc_volume
0,ANTs_SL_reg,0.037153,0.031298,84.242829,100.00000,14.897966
1,ANTs_SL_gauss,0.047003,0.038652,82.233787,100.00000,14.897966
2,ANTs_SL_lancz,0.042190,0.035423,83.961040,100.00000,14.897966
3,ANTs_SL_splines,0.041255,0.034414,83.418321,100.00000,14.897966
4,ANTs_t1w_reg,0.062901,0.034254,54.457968,89.20926,18.886478
5,ANTs_t1w_gauss,0.111262,0.051932,46.675099,89.20926,18.886478
6,ANTs_t1w_lancz,0.080985,0.033880,41.835420,89.20926,18.886478
7,ANTs_t1w_splines,0.102826,0.033518,32.596547,89.20926,18.886478
8,ANTs_walnut_reg,0.044889,0.011284,25.137363,71.41374,14.234087
9,ANTs_walnut_gauss,0.091228,0.028299,31.020122,71.41374,14.234087


In [9]:
print(df.to_latex(index=False)) 

\begin{tabular}{lrrrrr}
\toprule
                name &    error &  error\_in\_borders &  percentage &  perc\_gradient &  perc\_volume \\
\midrule
         ANTs\_SL\_reg & 0.037153 &          0.031298 &   84.242829 &      100.00000 &    14.897966 \\
       ANTs\_SL\_gauss & 0.047003 &          0.038652 &   82.233787 &      100.00000 &    14.897966 \\
       ANTs\_SL\_lancz & 0.042190 &          0.035423 &   83.961040 &      100.00000 &    14.897966 \\
     ANTs\_SL\_splines & 0.041255 &          0.034414 &   83.418321 &      100.00000 &    14.897966 \\
        ANTs\_t1w\_reg & 0.062901 &          0.034254 &   54.457968 &       89.20926 &    18.886478 \\
      ANTs\_t1w\_gauss & 0.111262 &          0.051932 &   46.675099 &       89.20926 &    18.886478 \\
      ANTs\_t1w\_lancz & 0.080985 &          0.033880 &   41.835420 &       89.20926 &    18.886478 \\
    ANTs\_t1w\_splines & 0.102826 &          0.033518 &   32.596547 &       89.20926 &    18.886478 \\
     ANTs\_walnut\_reg & 0.04

In [10]:
df.to_excel("pointwise_oversampling_DSSIM.xlsx", index=False)